# Thêm các thư viện

In [7]:
from graphframes import *
from pyspark.storagelevel import StorageLevel
from pyspark.sql.functions import col
import os
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("GraphAnalysis") \
    .getOrCreate()


# Đọc dữ liệu từ file CSV

* Đọc thông tin người, là đỉnh của đồ thị

In [8]:
v = spark.read.csv("data/2_people_data_2k.csv", header=True, inferSchema=True)

* Đọc thông tin quan hệ giữa mọi người, là cạnh của đồ thị

In [9]:
e = spark.read.csv("data/2_connections_data_300k.csv", header=True, inferSchema=True)

# Tạo đồ thị bằng thư viện GraphFrames

### Tạo GraphFrame

> * Người dùng có thể tạo GraphFrames từ các DataFrame đỉnh và cạnh.
>> * **DataFrame đỉnh (Vertex DataFrame):** Một DataFrame đỉnh cần chứa một cột đặc biệt có tên là "id", chỉ định các ID duy nhất cho mỗi đỉnh trong đồ thị.
>> * **DataFrame cạnh (Edge DataFrame):** Một DataFrame cạnh cần chứa hai cột đặc biệt: "src" (ID đỉnh nguồn của cạnh) và "dst" (ID đỉnh đích của cạnh).
>
> * Cả hai DataFrame đều có thể có các cột tùy ý khác. Các cột này có thể đại diện cho các thuộc tính của đỉnh và cạnh.
> * Một GraphFrame cũng có thể được xây dựng từ một DataFrame duy nhất chứa thông tin về cạnh. Các đỉnh sẽ được suy ra từ các nguồn và đích của các cạnh.

In [10]:
g = GraphFrame(v, e)
g.persist(StorageLevel.MEMORY_AND_DISK)

/usr/local/spark/python/pyspark/sql/dataframe.py:169: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


GraphFrame(v:[id: int, name: string ... 1 more field], e:[src: int, dst: int ... 1 more field])

### Hiển thị tập đỉnh và cạnh dưới dạng Dataframe

In [11]:
print("GraphFrame được tạo thành công:")
g.vertices.show()
g.edges.show()

GraphFrame được tạo thành công:
+---+-------------+---+
| id|         name|age|
+---+-------------+---+
|  1|Phạm Thế Châu| 41|
|  2|  Hoàng Nhiên| 24|
|  3|      Lê Thảo| 46|
|  4|  Trần Duy Hà| 56|
|  5|    Mai Hương| 35|
|  6|       Vũ Hải| 61|
|  7|      Đặng Hà| 32|
|  8|      Vũ Thảo| 49|
|  9|    Trần Khoa| 77|
| 10|    Nguyễn An| 27|
| 11|    Phạm Phúc| 68|
| 12|      Bùi Mai| 78|
| 13|Đặng Duy Dũng| 39|
| 14|   Vũ Trí Hải| 30|
| 15|     Hoàng Hà| 55|
| 16|       Vũ Kim| 24|
| 17|      Lê Hạnh| 38|
| 18|  Nguyễn Châu| 29|
| 19|Nguyễn Bảo Vũ| 64|
| 20|      Vũ Linh| 65|
+---+-------------+---+
only showing top 20 rows

+----+----+------------+
| src| dst|relationship|
+----+----+------------+
|1770|1228|      friend|
|1405| 541|      friend|
| 309| 904|      friend|
|   1| 751|      follow|
| 968| 540|      follow|
|1840| 469|      follow|
|   4|1022|      follow|
| 498|1557|      follow|
|1267| 607|      friend|
|1277|1295|      follow|
|1738| 192|      friend|
| 406|1968|     

# Khám phá GraphFrames cơ bản

### Hiển thị số lượng đỉnh và cạnh

In [12]:
print(f"Số lượng đỉnh: {g.vertices.count()}")
print(f"Số lượng cạnh: {g.edges.count()}")

Số lượng đỉnh: 2000
Số lượng cạnh: 300000


### Hiển thị bậc vào (in-degree) của mỗi đỉnh

In [13]:
print("Bậc vào của các đỉnh:")
g.inDegrees.show()

Bậc vào của các đỉnh:


/usr/local/spark/python/pyspark/sql/dataframe.py:148: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


+----+--------+
|  id|inDegree|
+----+--------+
| 471|     159|
|1591|     137|
| 463|     147|
|1829|     145|
| 833|     154|
|1342|     160|
|1238|     176|
| 496|     139|
|1088|     163|
| 148|     148|
|1959|     153|
|1645|     143|
|1580|     140|
| 540|     164|
|1990|     183|
| 392|     129|
|1896|     160|
| 897|     162|
|1084|     127|
| 243|     159|
+----+--------+
only showing top 20 rows



### Hiển thị bậc ra (out-degree) của mỗi đỉnh

In [14]:
print("Bậc ra của các đỉnh:")
g.outDegrees.show()

Bậc ra của các đỉnh:
+----+---------+
|  id|outDegree|
+----+---------+
|1088|      157|
| 833|      140|
|1580|      161|
|1342|      155|
|1645|      148|
|1959|      135|
| 463|      143|
|1829|      160|
| 496|      150|
| 148|      146|
| 471|      138|
|1238|      160|
|1591|      144|
|1721|      135|
|1395|      143|
| 623|      158|
|1507|      159|
|1025|      163|
| 737|      141|
|1522|      173|
+----+---------+
only showing top 20 rows



### Hiển thị tổng bậc (degree) của mỗi đỉnh

In [15]:
print("Tổng bậc của các đỉnh:")
g.degrees.show()

Tổng bậc của các đỉnh:
+----+------+
|  id|degree|
+----+------+
|1088|   320|
| 833|   294|
|1580|   301|
| 471|   297|
|1591|   281|
|1342|   315|
|1645|   291|
| 463|   290|
|1829|   305|
|1959|   288|
|1238|   336|
| 496|   289|
| 148|   294|
| 540|   313|
|1990|   310|
| 392|   278|
|1721|   285|
|1896|   303|
|1395|   286|
| 623|   307|
+----+------+
only showing top 20 rows



### Lọc các đỉnh theo thuộc tính (ví dụ: tuổi > 60)

In [16]:
print("Các đỉnh có tuổi lớn hơn 60:")
g.vertices.filter("age > 60").show()

Các đỉnh có tuổi lớn hơn 60:
+---+----------------+---+
| id|            name|age|
+---+----------------+---+
|  6|          Vũ Hải| 61|
|  9|       Trần Khoa| 77|
| 11|       Phạm Phúc| 68|
| 12|         Bùi Mai| 78|
| 19|   Nguyễn Bảo Vũ| 64|
| 20|         Vũ Linh| 65|
| 22|Dương Bảo Phương| 69|
| 24| Dương Duy Nhiên| 68|
| 32|      Vũ Mai Lâm| 69|
| 38|         Lê Hồng| 77|
| 40|     Nguyễn Nhật| 62|
| 42|  Trần Phú Thành| 77|
| 46|    Mai Phú Minh| 63|
| 47|          Lê Lâm| 71|
| 49|    Vũ Hữu Thành| 70|
| 53|     Phạm Phương| 68|
| 55|       Phạm Linh| 71|
| 59|        Trần Anh| 67|
| 60| Nguyễn Bảo Hạnh| 76|
| 65|         Bùi Huy| 71|
+---+----------------+---+
only showing top 20 rows



### Lọc các cạnh theo thuộc tính (ví dụ: relationship là 'friend')

In [17]:
print("Các cạnh có mối quan hệ 'friend':")
g.edges.filter("relationship = 'friend'").show()

Các cạnh có mối quan hệ 'friend':
+----+----+------------+
| src| dst|relationship|
+----+----+------------+
|1770|1228|      friend|
|1405| 541|      friend|
| 309| 904|      friend|
|1267| 607|      friend|
|1738| 192|      friend|
| 308|1433|      friend|
|1942|1990|      friend|
|1954| 756|      friend|
| 488|  88|      friend|
|1360|  17|      friend|
|1852| 479|      friend|
| 458|1377|      friend|
|  47| 345|      friend|
| 114|1567|      friend|
|1867|1511|      friend|
|1268| 428|      friend|
| 700| 910|      friend|
|1426|1768|      friend|
| 786| 684|      friend|
| 197| 289|      friend|
+----+----+------------+
only showing top 20 rows



# Truy vấn đồ thị (Graph Queries) - Tìm kiếm mẫu (Motif Finding)

### __Ví dụ 1:__ Tìm các cặp (a,b) sao cho a là bạn của b VÀ b là bạn của a
##### *__Lưu ý:__ Cần đảm bảo cột 'relationship' tồn tại và có giá trị 'friend'*
##### *Nếu không có cột relationship, bỏ điều kiện e.relationship = ...*

In [18]:
motifs = g.find("(a)-[e]->(b); (b)-[e2]->(a)") \
          .filter("e.relationship = 'friend' and e2.relationship = 'friend'")

In [19]:
print("Các cặp bạn bè tương hỗ (A->B và B->A):")
# Chọn lọc để tránh trùng lặp (ví dụ: chỉ hiển thị khi id a < id b)
mutual_friends = motifs.filter("a.id < b.id").dropDuplicates(['a', 'b'])
mutual_friends.select("a.name", "b.name").show()

Các cặp bạn bè tương hỗ (A->B và B->A):
+--------------+----------------+
|          name|            name|
+--------------+----------------+
|    Nguyễn Lan|        Mai Linh|
|   Phạm Thế An|        Phạm Kim|
|        Mai Tú|       Mai Trung|
|     Phạm Ngọc|      Nguyễn Nam|
|    Mai Văn An|       Đặng Dũng|
|   Nguyễn Châu|  Đặng Hải Trọng|
|Nguyễn Duy Lâm|      Phạm Thành|
|        Lê Vân|      Trần Thành|
|       Trần Hà|          Mai An|
|      Đặng Kim|       Lê Mai An|
|     Mai Quang|  Đặng Mai Hoàng|
|       Trần Hà|         Trần Vũ|
| Mai Xuân Hạnh|     Hoàng Thành|
|       Đặng Tú|    Vũ Đức Trung|
|   Lê Trí Tùng|Nguyễn Hữu Thành|
|      Đặng Hải|   Mai Bảo Nhiên|
| Đặng Mai Châu|         Đặng Tú|
|     Mai Nhiên| Nguyễn Phú Hưng|
|        Lê Kim|          Vũ Kim|
|   Nguyễn Thảo|       Dương Mai|
+--------------+----------------+
only showing top 20 rows



### __Ví dụ 2:__ Tìm "bạn của bạn" (Friends of friends - FoF)

##### *Tìm a -> b -> c, trong đó a không phải là c và a chưa phải là bạn trực tiếp của c*

In [20]:
fof = g.find("(a)-[e1]->(b); (b)-[e2]->(c)") \
          .filter("e1.relationship = 'friend' and e2.relationship = 'friend' and a.id != c.id") \
          .select(col('a.id').alias('a_id'), col('a.name').alias('a_name'), col('c.id').alias('c_id'), col('c.name').alias('c_name')) \
          .dropDuplicates(['a_id', 'c_id'])
fof.show()
fof.count()

+----+-------------+----+--------------+
|a_id|       a_name|c_id|        c_name|
+----+-------------+----+--------------+
|   1|Phạm Thế Châu|  25|     Nguyễn An|
|   1|Phạm Thế Châu|  37|     Phạm Tùng|
|   1|Phạm Thế Châu|  46|  Mai Phú Minh|
|   1|Phạm Thế Châu|  59|      Trần Anh|
|   1|Phạm Thế Châu|  95|Nguyễn Thế Anh|
|   1|Phạm Thế Châu|  96| Đặng Quang An|
|   1|Phạm Thế Châu| 102|Phạm Xuân Khoa|
|   1|Phạm Thế Châu| 114|       Lê Hạnh|
|   1|Phạm Thế Châu| 147|   Đặng Tấn Hà|
|   1|Phạm Thế Châu| 152|Đặng Đức Nhiên|
|   1|Phạm Thế Châu| 204|    Hoàng Phúc|
|   1|Phạm Thế Châu| 226|     Lê Mai An|
|   1|Phạm Thế Châu| 239|  Hoàng Thế An|
|   1|Phạm Thế Châu| 255|   Vũ Đức Hưng|
|   1|Phạm Thế Châu| 284|   Vũ Hải Hưng|
|   1|Phạm Thế Châu| 285|     Bùi Dương|
|   1|Phạm Thế Châu| 318| Vũ Hoàng Dũng|
|   1|Phạm Thế Châu| 329|   Trần Phương|
|   1|Phạm Thế Châu| 353|    Nguyễn Ánh|
|   1|Phạm Thế Châu| 372|     Trần Hưng|
+----+-------------+----+--------------+
only showing top

3757084

##### *Tìm những người đã là bạn trực tiếp*

In [21]:
direct_friends = g.find("(a)-[e]->(c)").filter("e.relationship = 'friend'") \
                      .select(col('a.id').alias('a_id'), col('a.name').alias('a_name'), col('c.id').alias('c_id'), col('c.name').alias('c_name'))
direct_friends.show()
direct_friends.count()

+----+---------------+----+----------------+
|a_id|         a_name|c_id|          c_name|
+----+---------------+----+----------------+
|1770|Nguyễn Duy Khoa|1228|   Mai Trí Thành|
|1405|     Nguyễn Bảo| 541|          Lê Anh|
| 309|        Đặng Tú| 904|Nguyễn Xuân Khoa|
|1267| Bùi Thế Phương| 607|        Vũ Trọng|
|1738|Đặng Hoàng Phúc| 192|  Trần Thế Trọng|
| 308|       Lê Thành|1433|     Vũ Đức Khoa|
|1942|     Dương Minh|1990|      Nguyễn Huy|
|1954| Dương Văn Khoa| 756|     Nguyễn Xuân|
| 488|Hoàng Văn Thành|  88| Nguyễn Hữu Dũng|
|1360|     Bùi Phương|  17|         Lê Hạnh|
|1852| Dương Bảo Phúc| 479| Trần Đức Phương|
| 458|         Bùi Hà|1377|         Lê Thảo|
|  47|         Lê Lâm| 345|           Vũ Hà|
| 114|        Lê Hạnh|1567|    Vũ Tấn Quang|
|1867|   Đặng Phú Hải|1511|          Vũ Bảo|
|1268|     Trần Thành| 428|   Hoàng Thế Nam|
| 700|        Trần An| 910|    Mai Duy Minh|
|1426|Hoàng Văn Nhiên|1768|    Mai Bảo Hạnh|
| 786|      Bùi Thành| 684|   Dương Văn Hải|
| 197|    

149848

##### *Loại bỏ những người đã là bạn trực tiếp khỏi danh sách FoF*

In [22]:
fof_results = fof.join(direct_friends, (fof.a_id == direct_friends.a_id) & (fof.c_id == direct_friends.c_id), "left_anti")
# left_anti là lấy phần trong tập bên trái không có trong tập bên phải
# fof_results.persist(StorageLevel.MEMORY_AND_DISK)

##### *Hiển thị kết quả*

In [23]:
print("Bạn của bạn (chưa phải bạn trực tiếp):")
fof_results.select('a_name', 'c_name').show()
fof_results.count()

Bạn của bạn (chưa phải bạn trực tiếp):
+-------------+--------------+
|       a_name|        c_name|
+-------------+--------------+
|Phạm Thế Châu|     Nguyễn An|
|Phạm Thế Châu|     Phạm Tùng|
|Phạm Thế Châu|  Mai Phú Minh|
|Phạm Thế Châu|      Trần Anh|
|Phạm Thế Châu|Nguyễn Thế Anh|
|Phạm Thế Châu| Đặng Quang An|
|Phạm Thế Châu|Phạm Xuân Khoa|
|Phạm Thế Châu|       Lê Hạnh|
|Phạm Thế Châu|   Đặng Tấn Hà|
|Phạm Thế Châu|Đặng Đức Nhiên|
|Phạm Thế Châu|    Hoàng Phúc|
|Phạm Thế Châu|     Lê Mai An|
|Phạm Thế Châu|  Hoàng Thế An|
|Phạm Thế Châu|   Vũ Đức Hưng|
|Phạm Thế Châu|   Vũ Hải Hưng|
|Phạm Thế Châu|     Bùi Dương|
|Phạm Thế Châu| Vũ Hoàng Dũng|
|Phạm Thế Châu|   Trần Phương|
|Phạm Thế Châu|    Nguyễn Ánh|
|Phạm Thế Châu|     Trần Hưng|
+-------------+--------------+
only showing top 20 rows



3616227

# Chạy các thuật toán đồ thị (Graph Algorithms)

## 1. PageRank - Đánh giá tầm quan trọng của các đỉnh

In [24]:
print("Chạy PageRank:")
# resetProbability là xác suất nhảy ngẫu nhiên (thường là 0.15)
# maxIter là số vòng lặp tối đa
results_pr = g.pageRank(resetProbability=0.15, maxIter=10)

Chạy PageRank:


In [25]:
# Hiển thị PageRank của các đỉnh
results_pr.vertices.select("id", "name", "pagerank").orderBy("pagerank", ascending=False).show()
# Hiển thị trọng số PageRank trên các cạnh (nếu cần)
results_pr.edges.select("src", "dst", "weight").show()

+----+------------+------------------+
|  id|        name|          pagerank|
+----+------------+------------------+
| 918|   Vũ Phương|1.2482510238869442|
| 270|      Vũ Bảo|1.2077985227994856|
| 614| Dương Hương|1.2015109600777394|
| 734|Hoàng Phú Hà|1.2012076362250472|
|1294|   Lê Văn Hà|1.1931187326217156|
| 167| Dương Thành| 1.191404408238185|
|1990|  Nguyễn Huy|1.1901550940384686|
| 294| Phạm Phương|1.1883077977798249|
|1689| Phạm Văn An|1.1816252532851101|
|1938| Mai Văn Bảo|1.1795471618219655|
|1994| Nguyễn Hồng|1.1760486319014916|
| 249|  Dương Linh|1.1756851366285395|
|1621|   Phạm Tùng|1.1754447668843968|
|1415|    Lê Hương|1.1753980440344365|
|1017|Hoàng Đức Hà|1.1718337718503957|
| 723|  Mai Văn Vũ|1.1717918967435907|
| 374|     Mai Kim|1.1698267563378117|
|1784|    Phạm Lâm|1.1672699168568725|
|1956|   Phạm Khoa| 1.165040173849614|
| 187|Vũ Tấn Thành|1.1635621416020636|
+----+------------+------------------+
only showing top 20 rows

+---+---+--------------------+
|src|ds

## 2. Connected Components - Tìm các nhóm đỉnh liên thông với nhau

In [27]:
# Tạo thư mục để lưu các checkpoints khi chạy thuật toán
!mkdir checkpoints
# Ghi nhận thông tin thư mục checkpoints vào ngữ cảnh Spark
spark.sparkContext.setCheckpointDir("checkpoints")

mkdir: cannot create directory ‘checkpoints’: File exists


In [28]:
print("Chạy Connected Components (tìm thành phần liên thông):")
# Thuật toán này gán một ID thành phần (component) cho mỗi đỉnh
results_cc = g.connectedComponents()
results_cc.select("id", "name", "component").orderBy("component").show()

Chạy Connected Components (tìm thành phần liên thông):
+---+-------------+---------+
| id|         name|component|
+---+-------------+---------+
|  1|Phạm Thế Châu|        1|
|  2|  Hoàng Nhiên|        1|
|  3|      Lê Thảo|        1|
|  4|  Trần Duy Hà|        1|
|  5|    Mai Hương|        1|
|  6|       Vũ Hải|        1|
|  7|      Đặng Hà|        1|
|  8|      Vũ Thảo|        1|
|  9|    Trần Khoa|        1|
| 10|    Nguyễn An|        1|
| 11|    Phạm Phúc|        1|
| 12|      Bùi Mai|        1|
| 13|Đặng Duy Dũng|        1|
| 14|   Vũ Trí Hải|        1|
| 15|     Hoàng Hà|        1|
| 16|       Vũ Kim|        1|
| 17|      Lê Hạnh|        1|
| 18|  Nguyễn Châu|        1|
| 19|Nguyễn Bảo Vũ|        1|
| 20|      Vũ Linh|        1|
+---+-------------+---------+
only showing top 20 rows



In [29]:
# On Windows:
!rmdir /s /q checkpoints

rmdir: failed to remove '/s': No such file or directory
rmdir: failed to remove '/q': No such file or directory
rmdir: failed to remove 'checkpoints': Directory not empty


## 3. Breadth-First Search (BFS) - Tìm đường đi ngắn nhất

In [30]:
print("Chạy BFS để tìm đường đi từ 'Vũ Bảo' (id=270) đến những người có 'age > 79', với mối liên hệ là follow")
# Tìm đường đi từ đỉnh có id=270 đến các đỉnh thỏa mãn điều kiện age > 79, với mối liên hệ là follow
paths = g.bfs(fromExpr="id = 270", toExpr="age > 79", edgeFilter="relationship = 'follow'", maxPathLength=5) # maxPathLength giới hạn độ dài đường đi

Chạy BFS để tìm đường đi từ 'Vũ Bảo' (id=270) đến những người có 'age > 79', với mối liên hệ là follow


In [31]:
print("Các đường đi tìm được:")
paths.show()
# Lưu ý: Kết quả paths chứa các cột e0, v1, e1, v2,... biểu diễn đường đi.

Các đường đi tìm được:
+-----------------+-------------------+--------------------+--------------------+--------------------+
|             from|                 e0|                  v1|                  e1|                  to|
+-----------------+-------------------+--------------------+--------------------+--------------------+
|{270, Vũ Bảo, 63}|{270, 1339, follow}|{1339, Hoàng Khoa...| {1339, 489, follow}|{489, Dương Quang...|
|{270, Vũ Bảo, 63}|{270, 1339, follow}|{1339, Hoàng Khoa...| {1339, 870, follow}|{870, Dương Mai, 80}|
|{270, Vũ Bảo, 63}|{270, 1339, follow}|{1339, Hoàng Khoa...|{1339, 1902, follow}|  {1902, Bùi An, 80}|
|{270, Vũ Bảo, 63}|{270, 1339, follow}|{1339, Hoàng Khoa...|{1339, 1609, follow}|{1609, Vũ Đức Bảo...|
|{270, Vũ Bảo, 63}|{270, 1339, follow}|{1339, Hoàng Khoa...|{1339, 1297, follow}| {1297, Lê Hạnh, 80}|
|{270, Vũ Bảo, 63}|  {270, 65, follow}|   {65, Bùi Huy, 71}|   {65, 109, follow}| {109, Mai Nhật, 80}|
|{270, Vũ Bảo, 63}| {270, 588, follow}| {588, Bùi 

In [32]:
paths.count()

125

## 4. Shortest Paths - Tìm đường đi ngắn nhất (yêu cầu chỉ định các đỉnh đích cụ thể)

In [33]:
print("Tìm đường đi ngắn nhất (là bạn bè) từ tất cả các đỉnh đến 'Bùi Ánh' (id = 1009):")
# landmarks là danh sách các đỉnh đích
results_sp = g.filterEdges("relationship = 'friend'").shortestPaths(landmarks=["1009"])
results_sp.select("id", "name", "distances").show()
# distances là một Map chứa khoảng cách từ đỉnh hiện tại đến các landmarks

Tìm đường đi ngắn nhất (là bạn bè) từ tất cả các đỉnh đến 'Bùi Ánh' (id = 1009):
+----+---------------+-----------+
|  id|           name|  distances|
+----+---------------+-----------+
|1084|        Mai Bảo|{1009 -> 2}|
|1410|    Dương Thành|{1009 -> 2}|
|1687|    Đặng Thế An|{1009 -> 2}|
| 667|    Vũ Hải Khoa|{1009 -> 1}|
|1053| Đặng Hoàng Lâm|{1009 -> 2}|
|1894| Vũ Hoàng Quang|{1009 -> 2}|
| 466|  Dương Xuân An|{1009 -> 2}|
|1040|      Đặng Thảo|{1009 -> 1}|
| 912|      Phạm Hồng|{1009 -> 2}|
|1325|     Trần Duyên|{1009 -> 2}|
| 140|         Vũ Hải|{1009 -> 2}|
| 204|     Hoàng Phúc|{1009 -> 1}|
| 956|Nguyễn Trí Khoa|{1009 -> 2}|
| 291|  Phạm Mai Tùng|{1009 -> 2}|
|   1|  Phạm Thế Châu|{1009 -> 2}|
| 931|   Phạm Văn Lâm|{1009 -> 2}|
|1466|   Nguyễn Thành|{1009 -> 2}|
| 755|         Lê Chi|{1009 -> 2}|
|1813|  Phạm Đức Khoa|{1009 -> 2}|
|1393|     Lê Xuân Vũ|{1009 -> 1}|
+----+---------------+-----------+
only showing top 20 rows



In [34]:
print("Lọc lại danh sách các đỉnh có khoảng cách đường đi ngắn nhất đến đỉnh 1009, lớn hơn 2:")
results_sp.filter(results_sp.distances['1009'] > 2).show()

Lọc lại danh sách các đỉnh có khoảng cách đường đi ngắn nhất đến đỉnh 1009, lớn hơn 2:
+----+--------------+---+-----------+
|  id|          name|age|  distances|
+----+--------------+---+-----------+
|1596|    Trần Trọng| 51|{1009 -> 3}|
|1792|Nguyễn Hải Bảo| 63|{1009 -> 3}|
| 702|   Phạm Thế Tú| 25|{1009 -> 3}|
|1290|     Dương Yến| 42|{1009 -> 3}|
|1004|      Phạm Ánh| 44|{1009 -> 3}|
| 171|      Trần Kim| 35|{1009 -> 3}|
|1070|    Lê Hải Bảo| 33|{1009 -> 3}|
| 165|  Mai Bảo Hưng| 53|{1009 -> 3}|
|1011|   Nguyễn Dũng| 63|{1009 -> 3}|
|1660|    Bùi Bảo Hà| 64|{1009 -> 3}|
| 743|   Hoàng Quang| 40|{1009 -> 3}|
|1899|      Đặng Anh| 49|{1009 -> 3}|
|   6|        Vũ Hải| 61|{1009 -> 3}|
| 475|      Bùi Dũng| 68|{1009 -> 3}|
| 426|        Lê Vân| 28|{1009 -> 3}|
|1616|   Vũ Hoàng Tú| 63|{1009 -> 3}|
|  17|       Lê Hạnh| 38|{1009 -> 3}|
| 861|       Trần Vũ| 29|{1009 -> 3}|
| 775|   Hoàng Hương| 38|{1009 -> 3}|
|1477|     Dương Ánh| 57|{1009 -> 3}|
+----+--------------+---+-----------+
o